In [1]:
import sys, os, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go


In [2]:
sys.path.append(
    "/Users/wiegerscheurer/repos/physicspred"
)  # To enable importing from repository folders

In [3]:
150/2.2




68.18181818181817

In [4]:
from functions.physics import _rotate_90, _dir_to_vec, _vec_to_dir, _flip_dir
from functions.utilities import determine_sequence
from functions.analysis import (get_precision, 
                                get_data, 
                                get_false_negatives,
                                get_false_positives,
                                get_true_positives,
                                get_true_negatives,
                                filter_condition,
                                get_f1_score, 
                                get_rt, 
                                get_accuracy,
                                get_hit_rate,)


### BALANCING THEM TRIALS

In [ ]:
interactor_trial_options = ["45_top_r", "45_top_u",
                            "45_bottom_l", "45_bottom_d",
                            "135_top_l", "135_top_u",
                            "135_bottom_r", "135_bottom_d"]
empty_trial_options = ["none_l", "none_r", "none_u", "none_d"]  # For the none trials


bounce_options = [True, False]
rand_bounce_direction_options = ["left", "right"] * 2
random.shuffle(rand_bounce_direction_options)


target_baserate = .5
ball_change_options = [True] * 1 + [False] * int((1 / target_baserate) - 1)
# rand_speed_change_options = ["slower", "faster"]
natural_speed_variance = .5 #config["natural_speed_variance"]
avg_ball_speed = 11

ball_speed_options = list(
    np.arange(
        avg_ball_speed - natural_speed_variance,
        avg_ball_speed + (2 * natural_speed_variance),
        natural_speed_variance,
    )
)



trial_types = determine_sequence(n_trials, [1, 0], randomised=True) # 1 is interactor, 0 is empty trial

interactor_trials = balance_over_bool(trial_types, interactor_trial_options, randomised=True)

# Create deterministically randomised; balanced parameter sequences
trials = determine_sequence(n_trials, trial_options, randomised=True)

bounces = determine_sequence(n_trials, bounce_options, randomised=True)

rand_bounce_directions = get_phantbounce_sequence(trials, rand_bounce_direction_options, randomised=True) # Random phantom bounce direction

ball_changes = determine_sequence(n_trials, ball_change_options, randomised=True)
rand_speed_changes = determine_sequence(
    n_trials, rand_speed_change_options, randomised=True
)
ball_speeds = determine_sequence(n_trials, ball_speed_options, randomised=True)

ball_spawn_spread = config[
    "ball_spawn_spread"
]  # Margin around fixation where the ball can spawn (smaller = )


In [6]:
import pandas as pd
import numpy as np
import random
from itertools import product

def create_balanced_trial_design(trial_n=None):
    # Your options
    interactor_trial_options = ["45_top_r", "45_top_u", "45_bottom_l", "45_bottom_d",
                               "135_top_l", "135_top_u", "135_bottom_r", "135_bottom_d"]
    empty_trial_options = ["none_l", "none_r", "none_u", "none_d"]
    bounce_options = [True, False]
    ball_change_options = [True, False]
    
    # Updated to 3 different ball speeds
    avg_ball_speed = 11
    natural_speed_variance = 0.5
    ball_speed_options = list(np.arange(
        avg_ball_speed - natural_speed_variance,
        avg_ball_speed + (2 * natural_speed_variance),
        natural_speed_variance
    ))[:3]  # Take only 3 speeds
    
    # Map each empty trial option to a specific bounce direction
    direction_mapping = {
        "none_l": "left",
        "none_r": "right",
        "none_u": "left",
        "none_d": "right"
    }
    
    # Calculate minimum trials for perfect balance with 3 ball speeds
    # For interactor: 8 options × 2 bounce × 2 ball_change × 3 speeds = 96 combinations
    # For empty: 4 options × 2 bounce × 2 ball_change × 3 speeds = 48 combinations
    # To keep interactor:empty balanced, we need 96 of each = 192 total minimum
    
    # If trial_n is specified, create a balanced subset
    if trial_n is not None:
        # Make sure trial_n is even for interactor:empty balance
        if trial_n % 2 == 1:
            trial_n -= 1
            print(f"Adjusted trial count to {trial_n} to maintain balance")
        
        half_n = trial_n // 2  # Half for interactor, half for empty
        
        # Create dataframe to store the balanced design
        all_trials = []
        
        # For interactor trials
        interactor_combos = list(product(
            interactor_trial_options,
            bounce_options,
            ball_change_options,
            ball_speed_options
        ))
        # Will need to repeat some combinations if half_n > len(interactor_combos)
        interactor_subset = [interactor_combos[i % len(interactor_combos)] for i in range(half_n)]
        random.shuffle(interactor_subset)
        
        # Create interactor trials
        for i in range(half_n):
            trial_option, bounce, ball_change, ball_speed = interactor_subset[i]
            all_trials.append({
                'trial_type': 'interactor',
                'trial_option': trial_option,
                'bounce': bounce,
                'phant_bounce_direction': None,
                'ball_change': ball_change,
                'ball_speed': ball_speed
            })
        
        # For empty trials
        empty_combos = list(product(
            empty_trial_options,
            bounce_options,
            ball_change_options,
            ball_speed_options
        ))
        # Will need to repeat some combinations if half_n > len(empty_combos)
        empty_subset = [empty_combos[i % len(empty_combos)] for i in range(half_n)]
        random.shuffle(empty_subset)
        
        # Create empty trials
        for i in range(half_n):
            trial_option, bounce, ball_change, ball_speed = empty_subset[i]
            bounce_direction = direction_mapping[trial_option] if bounce else None
            all_trials.append({
                'trial_type': 'empty',
                'trial_option': trial_option,
                'bounce': bounce,
                'phant_bounce_direction': bounce_direction,
                'ball_change': ball_change,
                'ball_speed': ball_speed
            })
        
        # Convert to dataframe and shuffle
        df = pd.DataFrame(all_trials)
        return df.sample(frac=1).reset_index(drop=True)
    
    # If trial_n is None, create the full balanced design
    else:
        # Create all possible combinations
        all_trials = []
        
        # For interactor trials
        for combo in product(interactor_trial_options, bounce_options, ball_change_options, ball_speed_options):
            trial_option, bounce, ball_change, ball_speed = combo
            all_trials.append({
                'trial_type': 'interactor',
                'trial_option': trial_option,
                'bounce': bounce,
                'phant_bounce_direction': None,
                'ball_change': ball_change,
                'ball_speed': ball_speed
            })
        
        # For empty trials - we need to duplicate these to match interactor count
        for combo in product(empty_trial_options, bounce_options, ball_change_options, ball_speed_options):
            trial_option, bounce, ball_change, ball_speed = combo
            bounce_direction = direction_mapping[trial_option] if bounce else None
            
            # Each empty trial combination needs to appear twice to balance with interactor trials
            for _ in range(2):
                all_trials.append({
                    'trial_type': 'empty',
                    'trial_option': trial_option,
                    'bounce': bounce,
                    'bounce_direction': bounce_direction,
                    'ball_change': ball_change,
                    'ball_speed': ball_speed
                })
        
        # Convert to dataframe and shuffle
        df = pd.DataFrame(all_trials)
        return df.sample(frac=1).reset_index(drop=True)

def check_balance(df):
    print(f"Total trials: {len(df)}")
    
    # Check trial type balance
    type_counts = df['trial_type'].value_counts()
    print("\nTrial type balance:")
    print(type_counts)
    
    # Check trial option balance within each trial type
    print("\nTrial option balance for interactor trials:")
    interactor_options = df[df['trial_type'] == 'interactor']['trial_option'].value_counts().sort_index()
    print(interactor_options)
    print(f"Variance: {interactor_options.var():.2f}")
    
    print("\nTrial option balance for empty trials:")
    empty_options = df[df['trial_type'] == 'empty']['trial_option'].value_counts().sort_index()
    print(empty_options)
    print(f"Variance: {empty_options.var():.2f}")
    
    # Check bounce balance
    bounce_counts = df['bounce'].value_counts()
    print("\nBounce balance:")
    print(bounce_counts)
    
    # Check ball change balance
    ball_change_counts = df['ball_change'].value_counts()
    print("\nBall change balance:")
    print(ball_change_counts)
    
    # Check ball speed balance
    print("\nBall speed balance:")
    print(df['ball_speed'].value_counts().sort_index())
    
    # Cross-tabulations for more detailed balance checks
    print("\nCross-tabulation of trial_type × bounce:")
    print(pd.crosstab(df['trial_type'], df['bounce']))
    
    print("\nCross-tabulation of trial_type × ball_change:")
    print(pd.crosstab(df['trial_type'], df['ball_change']))
    
    print("\nCross-tabulation of bounce × ball_change:")
    print(pd.crosstab(df['bounce'], df['ball_change']))
    
    # Check balance at the deepest level
    print("\nBalance within interactor trial types:")
    for option in sorted(df[df['trial_type'] == 'interactor']['trial_option'].unique()):
        subset = df[(df['trial_type'] == 'interactor') & (df['trial_option'] == option)]
        print(f"\n{option}:")
        print(f"  Total: {len(subset)}")
        print(f"  Bounce: {subset['bounce'].value_counts().to_dict()}")
        print(f"  Ball Change: {subset['ball_change'].value_counts().to_dict()}")
        print(f"  Ball Speed: {subset['ball_speed'].value_counts().to_dict()}")
    
    print("\nBalance within empty trial types:")
    for option in sorted(df[df['trial_type'] == 'empty']['trial_option'].unique()):
        subset = df[(df['trial_type'] == 'empty') & (df['trial_option'] == option)]
        print(f"\n{option}:")
        print(f"  Total: {len(subset)}")
        print(f"  Bounce: {subset['bounce'].value_counts().to_dict()}")
        print(f"  Ball Change: {subset['ball_change'].value_counts().to_dict()}")
        print(f"  Ball Speed: {subset['ball_speed'].value_counts().to_dict()}")

# Calculate minimum trials for perfect balance
interactor_combos = 8 * 2 * 2 * 3  # 8 options × 2 bounce × 2 ball_change × 3 speeds = 96
empty_combos = 4 * 2 * 2 * 3  # 4 options × 2 bounce × 2 ball_change × 3 speeds = 48
min_perfect_balance = max(interactor_combos, empty_combos * 2)  # 96 interactor, 96 empty = 192 total

print(f"Minimum trials needed for perfect balance: {min_perfect_balance}")

# Example usage with different trial numbers
for trial_n in [24, 48, 96, 192]:
    print(f"\n\n=== TESTING WITH {trial_n} TRIALS ===")
    trial_df = create_balanced_trial_design(trial_n=trial_n)
    check_balance(trial_df)

Minimum trials needed for perfect balance: 96


=== TESTING WITH 24 TRIALS ===
Total trials: 24

Trial type balance:
trial_type
empty         12
interactor    12
Name: count, dtype: int64

Trial option balance for interactor trials:
trial_option
45_top_r    12
Name: count, dtype: int64
Variance: nan

Trial option balance for empty trials:
trial_option
none_l    12
Name: count, dtype: int64
Variance: nan

Bounce balance:
bounce
False    12
True     12
Name: count, dtype: int64

Ball change balance:
ball_change
True     12
False    12
Name: count, dtype: int64

Ball speed balance:
ball_speed
10.5    8
11.0    8
11.5    8
Name: count, dtype: int64

Cross-tabulation of trial_type × bounce:
bounce      False  True 
trial_type              
empty           6      6
interactor      6      6

Cross-tabulation of trial_type × ball_change:
ball_change  False  True 
trial_type               
empty            6      6
interactor       6      6

Cross-tabulation of bounce × ball_change:
ball_change

In [7]:

# load in csv file as pd.dataframe
df = pd.read_csv('/Users/wiegerscheurer/repos/physicspred/data/sub-willem/ball_hue/design_matrix.csv')

check_balance(df)

Total trials: 24

Trial type balance:
trial_type
empty         12
interactor    12
Name: count, dtype: int64

Trial option balance for interactor trials:
trial_option
45_top_r    12
Name: count, dtype: int64
Variance: nan

Trial option balance for empty trials:
trial_option
none_l    12
Name: count, dtype: int64
Variance: nan

Bounce balance:
bounce
False    12
True     12
Name: count, dtype: int64

Ball change balance:
ball_change
True     12
False    12
Name: count, dtype: int64

Ball speed balance:
ball_speed
10.5    8
11.0    8
11.5    8
Name: count, dtype: int64

Cross-tabulation of trial_type × bounce:
bounce      False  True 
trial_type              
empty           6      6
interactor      6      6

Cross-tabulation of trial_type × ball_change:
ball_change  False  True 
trial_type               
empty            6      6
interactor       6      6

Cross-tabulation of bounce × ball_change:
ball_change  False  True 
bounce                   
False            6      6
True        

#### Acquire data per sub

In [50]:
all_sub_names = ["wolpert", "wunger","scheur", "hendrik", "willem", "ikzelf", "melvin", "paulo", "yifan", "ann", "qifei", "mingyao", "bilge", "yanni", "eva"]
sub_stack = pd.DataFrame()

for sub_idx, sub in enumerate(all_sub_names):
    this_sub = get_data(subject=f"sub-{sub}", task="ball_hue")
    sub_stack = pd.concat([sub_stack, this_sub])
    
sub_stack.reset_index(inplace=True, drop=True)
# sub_stack

In [51]:
this_sub = random.choice(all_sub_names)
print(this_sub)
sub = get_data(subject=f"sub-{this_sub}", task="ball_hue")

simc_xpolc = get_rt(df=sub, sim_con=True, expol_con=True, return_df=True)["rt"]
simc_xpoli = get_rt(df=sub, sim_con=True, expol_con=False, return_df=True)["rt"]
simi_xpolc = get_rt(df=sub, sim_con=False, expol_con=True, return_df=True)["rt"]
simi_xpoli = get_rt(df=sub, sim_con=False, expol_con=False, return_df=True)["rt"]


all_kinds = [simc_xpolc, simc_xpoli, simi_xpolc, simi_xpoli]

for kind in all_kinds:
    print(len(kind))
    print(kind.mean(), kind.std())


willem
3
0.4469702083539839 0.017712759192814186
3
0.42305147224881995 0.012704657845265744
3
0.4377223887713626 0.01365834507879117
3
nan nan


#### Integrate in pilot script

In [52]:
sim_cons = [True, False]
expol_cons = [True, False]
# randsub = random.choice(all_sub_names)
randsub = "wolpert"
print(f"Now looking at : {randsub}")
for sim_con in sim_cons:
    
    for expol_con in expol_cons:
        
        print(f"sim_con: {sim_con}, expol_con: {expol_con}")
        dat = get_data(subject=f"sub-{randsub}", task="ball_hue")
        dat_filt = dat[(dat["sim_congruent"] == sim_con) & (dat["abs_congruent"] == expol_con)]
        print(len(dat_filt))

Now looking at : wolpert
sim_con: True, expol_con: True
6
sim_con: True, expol_con: False
6
sim_con: False, expol_con: True
6
sim_con: False, expol_con: False
6


In [53]:
wie = get_data(subject="sub-wolpert", task="ball_hue")

n_targets = len(wie[wie["ball_change"] == 1]) # 20
n_nones = len(wie[wie["interactor"] == "none_d"])# 20
n_nones

# sort dataframe based on trials column
wie.sort_values(by="interactor", inplace=True)

wie["response"]

23      NaN
2     right
4      down
5       NaN
6       NaN
19     down
8      down
9     right
18      NaN
17      NaN
16    right
15     down
21     down
20      NaN
0      left
13     down
12     down
22      NaN
10      NaN
7      down
3       NaN
1       NaN
14     left
11      NaN
Name: response, dtype: object

In [45]:
wie

,trial_type,trial,ball_speed,interactor,bounce,bounce_moment,random_bounce_direction,target_onset,ball_change,abs_congruent,...,start_pos,end_pos,abs_rfup,abs_rfright,abs_rfdown,abs_rfleft,sim_rfup,sim_rfright,sim_rfdown,sim_rfleft
0,interactor,1,5.50,45_top_r,False,NaN,NaN,NaN,False,True,...,left,right,"(0, 0)","(1, 1)","(0, 0)","(0, 0)","(0, 0)","(0, 1)","(1, 0)","(0, 0)"
21,interactor,22,5.75,45_top_r,False,NaN,NaN,NaN,False,True,...,left,right,"(0, 0)","(1, 1)","(0, 0)","(0, 0)","(0, 0)","(0, 1)","(1, 0)","(0, 0)"
18,interactor,19,5.25,45_top_r,False,NaN,NaN,NaN,False,True,...,left,right,"(0, 0)","(1, 1)","(0, 0)","(0, 0)","(0, 0)","(0, 1)","(1, 0)","(0, 0)"
16,interactor,17,5.25,45_top_r,False,NaN,NaN,3.659155,True,True,...,left,right,"(0, 0)","(1, 1)","(0, 0)","(0, 0)","(0, 0)","(0, 1)","(1, 0)","(0, 0)"
13,interactor,14,5.75,45_top_r,True,3.441456,NaN,NaN,False,False,...,left,down,"(0, 0)","(1, 0)","(0, 1)","(0, 0)","(0, 0)","(0, 0)","(1, 1)","(0, 0)"
22,interactor,23,5.25,45_top_r,True,3.424997,NaN,NaN,False,False,...,left,down,"(0, 0)","(1, 0)","(0, 1)","(0, 0)","(0, 0)","(0, 0)","(1, 1)","(0, 0)"
9,interactor,10,5.75,45_top_r,False,NaN,NaN,3.658901,True,True,...,left,right,"(0, 0)","(1, 1)","(0, 0)","(0, 0)","(0, 0)","(0, 1)","(1, 0)","(0, 0)"
8,interactor,9,5.50,45_top_r,True,3.411605,NaN,3.659903,True,False,...,left,down,"(0, 0)","(1, 0)","(0, 1)","(0, 0)","(0, 0)","(0, 0)","(1, 1)","(0, 0)"
23,interactor,24,5.75,45_top_r,True,3.422854,NaN,3.671786,True,False,...,left,down,"(0, 0)","(1, 0)","(0, 1)","(0, 0)","(0, 0)","(0, 0)","(1, 1)","(0, 0)"
5,interactor,6,5.50,45_top_r,True,3.437862,NaN,NaN,False,False,...,left,down,"(0, 0)","(1, 0)","(0, 1)","(0, 0)","(0, 0)","(0, 0)","(1, 1)","(0, 0)"


In [54]:

hit_rate = get_hit_rate(wie, False, False)
print(hit_rate)
wie_filt = filter_condition(wie, False, False)

fp = get_false_positives(wie_filt)
fn = get_false_negatives(wie_filt)
tp = get_true_positives(wie_filt)
tn = get_true_negatives(wie_filt)


print(f"False positives: {fp}")
print(f"False negatives: {fn}")
print(f"True positives: {tp}")
print(f"True negatives: {tn}")


hr = (tp) / (tp + fn)

hr
# get_misses(filter_condition(wie, False, False))

1.0
False positives: 1
False negatives: 0
True positives: 3
True negatives: 2


1.0

In [55]:
for sim_con in [True, False]:
    for xpol_con in [True, False]:
        print(f"sim_con: {sim_con}, xpol_con: {xpol_con}")
        print(f"{get_hit_rate(wie, sim_con, xpol_con):.2f}\n")

sim_con: True, xpol_con: True
0.67

sim_con: True, xpol_con: False
1.00

sim_con: False, xpol_con: True
1.00

sim_con: False, xpol_con: False
1.00



In [56]:
cc = filter_condition(wie, True, True)
ci = filter_condition(wie, True, False)
ic = filter_condition(wie, False, True)
ii = filter_condition(wie, False, False)

cc_targets = cc[cc["ball_change"] == 1]
ci_targets = ci[ci["ball_change"] == 1]
ic_targets = ic[ic["ball_change"] == 1]
ii_targets = ii[ii["ball_change"] == 1]

print(f"cc targets: {len(cc_targets)}")
print(f"ci targets: {len(ci_targets)}")
print(f"ic targets: {len(ic_targets)}")
print(f"ii targets: {len(ii_targets)}")

print(f"cc: {len(cc)}")
print(f"ci: {len(ci)}")
print(f"ic: {len(ic)}")
print(f"ii: {len(ii)}")


cc targets: 3
ci targets: 3
ic targets: 3
ii targets: 3
cc: 6
ci: 6
ic: 6
ii: 6


In [41]:
1/120

0.008333333333333333

In [57]:
for this_name in all_sub_names:
    wie = get_data(subject=f"sub-{this_name}", task="ball_hue")
    
    # Get hit rates for each condition
    simc_xpolc = get_hit_rate(df=wie, sim_con=True, expol_con=True, return_df=True)["accuracy"]
    simc_xpoli = get_hit_rate(df=wie, sim_con=True, expol_con=False, return_df=True)["accuracy"]
    simi_xpolc = get_hit_rate(df=wie, sim_con=False, expol_con=True, return_df=True)["accuracy"]
    simi_xpoli = get_hit_rate(df=wie, sim_con=False, expol_con=False, return_df=True)["accuracy"]
    
    cc_rt = get_rt(df=wie, sim_con=True, expol_con=True, return_df=False)
    ci_rt = get_rt(df=wie, sim_con=True, expol_con=False, return_df=False)
    ic_rt = get_rt(df=wie, sim_con=False, expol_con=True, return_df=False)
    ii_rt = get_rt(df=wie, sim_con=False, expol_con=False, return_df=False)

    print(f"Subject missed {get_false_negatives(wie)} trials")
    # print(f"simc_xpolc: {simc_xpolc.mean()}, simc_xpoli: {simc_xpoli.mean()}, simi_xpolc: {simi_xpolc.mean()}, simi_xpoli: {simi_xpoli.mean()}")

    # Create data for the 2x2 heatmap
    z_values = [[simc_xpolc.mean(), simc_xpoli.mean()], 
                [simi_xpolc.mean(), simi_xpoli.mean()]]
    
    # Create a figure with subplot for heatmap
    fig = go.Figure()

    # Add heatmap
    fig.add_trace(go.Heatmap(
        z=z_values,
        x=['Congruent', 'Incongruent'],
        y=['Congruent', 'Incongruent'],
        text=[[f"{simc_xpolc.mean():.3f}<br>#trials: {len(simc_xpolc)}<br>rt: {cc_rt:.2f}s", f"{simc_xpoli.mean():.3f}<br>#trials: {len(simc_xpoli)}<br>rt: {ci_rt:.2f}s"], 
            [f"{simi_xpolc.mean():.3f}<br>#trials: {len(simi_xpolc)}<br>rt: {ic_rt:.2f}s", f"{simi_xpoli.mean():.3f}<br>#trials: {len(simi_xpoli)}<br>rt: {ii_rt:.2f}s"]],
        texttemplate="%{text}",
        textfont={"size":14},
        colorscale="magma_r",
        showscale=True,
        colorbar=dict(title="Hit Rate"),
        zmin=0,  # Set minimum value for the color scale
        zmax=1   # Set maximum value for the color scale
    ))

    # Update layout
    fig.update_layout(
        title=f"Hit Rate by Condition - Subject {this_name}",
        xaxis=dict(title="Motion Extrapolation", side="bottom"),
        yaxis=dict(title="Simulation"),
        width=600,
        height=500
    )

    # Show the figure
    fig.show()

Subject missed 1 trials


Subject missed 3 trials


Subject missed 0 trials


Subject missed 3 trials


Subject missed 3 trials


...

Subject missed 16 trials


Subject missed 20 trials


Subject missed 14 trials


Subject missed 13 trials


Subject missed 36 trials


In [83]:

# Create an empty list to store all data frames
all_data = []

# Define types and their colors
types = ['Simcon + Xpolcon', 'Simcon + Xpolinc', 'Siminc + Xpolcon', 'Siminc + Xpolinc']

# Generate a colormap
cmap = plt.get_cmap('YlOrRd')  # You can choose a different colormap if you prefer
col_factor = 50
col_icept = 150
# Create the colors dictionary
type_colors = {name: f'rgba({int(cmap(i*col_factor + col_icept)[0]*255)}, {int(cmap(i*col_factor + col_icept)[1]*255)}, {int(cmap(i*col_factor + col_icept)[2]*255)}, 0.5)' 
          for i, name in enumerate(types)}

# Loop through all subjects (using the same structure as your original code)
for sub_name in all_sub_names:
# for sub_name in ["eva"]:
    sub = get_data(subject=f"sub-{sub_name}", task="ball_hue")
    
    simc_xpolc = get_rt(df=sub, sim_con=True, expol_con=True, return_df=True)["rt"]
    simc_xpoli = get_rt(df=sub, sim_con=True, expol_con=False, return_df=True)["rt"]
    simi_xpolc = get_rt(df=sub, sim_con=False, expol_con=True, return_df=True)["rt"]
    simi_xpoli = get_rt(df=sub, sim_con=False, expol_con=False, return_df=True)["rt"]
    
    # Create DataFrames without subject column - we'll aggregate across all subjects
    simc_xpolc_df = pd.DataFrame({
        'Reaction Time': simc_xpolc,
        'Type': 'Simcon + Xpolcon'
    })
    
    simc_xpoli_df = pd.DataFrame({
        'Reaction Time': simc_xpoli,
        'Type': 'Simcon + Xpolinc'
    })
    
    simi_xpolc_df = pd.DataFrame({
        'Reaction Time': simi_xpolc,
        'Type': 'Siminc + Xpolcon'
    })
    
    simi_xpoli_df = pd.DataFrame({
        'Reaction Time': simi_xpoli,
        'Type': 'Siminc + Xpolinc'
    })
    
    # Append all DataFrames to our list
    all_data.extend([simc_xpolc_df, simc_xpoli_df, simi_xpolc_df, simi_xpoli_df])

# Combine all the data into a single DataFrame
combined_data = pd.concat(all_data, ignore_index=True)

# Calculate mean reaction time for each type
type_means = combined_data.groupby('Type')['Reaction Time'].mean().to_dict()

# Create the plot
fig = go.Figure()

# Add violin plots for each type
for i, type_name in enumerate(types):
    subset = combined_data[combined_data['Type'] == type_name]
    
    fig.add_trace(go.Violin(
        x=[type_name] * len(subset),  # Use type name directly as x value
        y=subset['Reaction Time'],
        name=type_name,
        legendgroup=type_name,
        showlegend=True,
        box_visible=True,
        meanline_visible=True,
        points='all',
        jitter=0.2,
        pointpos=0.5,
        line_color=type_colors[type_name],
        side='negative',
        width=.5,
        spanmode='soft'
    ))

# Connect the means with a line
fig.add_trace(go.Scatter(
    x=types,
    y=[type_means[t] for t in types],
    mode='lines+markers',
    line=dict(color='rgba(0, 0, 0, 0.7)', width=8),
    marker=dict(
        size=14,  # Increased marker size
        color=[type_colors[t].replace('0.5', '1') for t in types],
        line=dict(color='black', width=4)
    ),
    name='Mean RT',
    hovertemplate='Mean: %{y:.3f}s<extra></extra>'
))

# Define a common font style for consistent text appearance
font_style = dict(
    family="Arial, sans-serif",
    size=20,  # Larger font size
    color="black"
)

# Update layout with enhanced text styling
fig.update_layout(
    title=dict(
        text="Average Reaction Times Across All Subjects\n\n",
        font=dict(size=25, family="Arial, sans-serif", color="black"),
        x=0.5,
        y=.985
    ),
    yaxis=dict(
        title=dict(
            text="Reaction Time (s)",
            font=font_style
        ),
        tickfont=font_style,
        tickwidth=2,
        showline=True,
        linewidth=3,
        linecolor='black'
    ),
    xaxis=dict(
        title=dict(
            text="Response Type",
            font=font_style
        ),
        tickmode='array',
        tickvals=types,
        tickfont=font_style,
        tickwidth=2,
        showline=True,
        linewidth=3,
        linecolor='black'
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.01,
        xanchor="center",
        x=0.5,
        font=dict(size=18, family="Arial, sans-serif"),
        borderwidth=2,
        bordercolor="White",
        title=dict(
            text="Response Type",
            font=dict(size=18, family="Arial, sans-serif")
        )
    ),
    violinmode='overlay',
    # height=1200,
    # width=1200,
    height=1200,
    width=1200,
    font=font_style,  # Default font for all other text elements
    plot_bgcolor='white',
    paper_bgcolor='white'
)

# Add annotations for the mean values
for i, type_name in enumerate(types):
    fig.add_annotation(
        x=type_name,
        y=type_means[type_name],
        text=f"{type_means[type_name]:.3f}s",
        showarrow=True,
        arrowhead=2,
        arrowsize=1.5,  # Slightly larger arrowhead
        arrowwidth=3,    # Thicker arrow
        arrowcolor='black',
        ax=120,
        ay=-90,
        font=dict(size=25, color='black', family="Arial, sans-serif")
    )

# Make the axis lines thicker
fig.update_xaxes(mirror=True, ticks='outside', tickwidth=3, ticklen=10, showgrid=False)
fig.update_yaxes(mirror=True, ticks='outside', tickwidth=3, ticklen=10, showgrid=True, gridwidth=1, gridcolor='lightgray')

fig.show()




In [34]:

# Create an empty list to store all data frames
all_data = []

# Define types and their colors
types = ['Abstraction', 'Simulation', 'Both']

# Generate a colormap
cmap = plt.get_cmap('YlOrRd')  # You can choose a different colormap if you prefer
col_factor = 50
col_icept = 150
# Create the colors dictionary
type_colors = {name: f'rgba({int(cmap(i*col_factor + col_icept)[0]*255)}, {int(cmap(i*col_factor + col_icept)[1]*255)}, {int(cmap(i*col_factor + col_icept)[2]*255)}, 0.5)' 
          for i, name in enumerate(types)}

# Loop through all subjects (using the same structure as your original code)
for sub_name in all_sub_names:
    sub = get_data(subject=f"sub-{sub_name}", task="ball_hue")
    # Get your data as before
    rt_dubs = get_rt(sub, hypothesis="both", include_dubtrials="only", return_df=True)
    rt_separate = get_rt(sub, hypothesis="both", include_dubtrials=False, return_df=True)
    sim_rt = rt_separate["simulation"]["rt"]
    abs_rt = rt_separate["abstraction"]["rt"]
    both_rt = rt_dubs["sim + abs"]["rt"]
    
    # Create DataFrames without subject column - we'll aggregate across all subjects
    sim_df = pd.DataFrame({
        'Reaction Time': sim_rt,
        'Type': 'Simulation'
    })
    
    abs_df = pd.DataFrame({
        'Reaction Time': abs_rt,
        'Type': 'Abstraction'
    })
    
    both_df = pd.DataFrame({
        'Reaction Time': both_rt,
        'Type': 'Both'
    })
    
    # Append all DataFrames to our list
    all_data.extend([abs_df, sim_df, both_df])

# Combine all the data into a single DataFrame
combined_data = pd.concat(all_data, ignore_index=True)

# Calculate mean reaction time for each type
type_means = combined_data.groupby('Type')['Reaction Time'].mean().to_dict()

# Create the plot
fig = go.Figure()

# Add violin plots for each type
for i, type_name in enumerate(types):
    subset = combined_data[combined_data['Type'] == type_name]
    
    fig.add_trace(go.Violin(
        x=[type_name] * len(subset),  # Use type name directly as x value
        y=subset['Reaction Time'],
        name=type_name,
        legendgroup=type_name,
        showlegend=True,
        box_visible=True,
        meanline_visible=True,
        points='all',
        jitter=0.2,
        pointpos=0.5,
        line_color=type_colors[type_name],
        side='negative',
        width=.5,
        spanmode='soft'
    ))

# Connect the means with a line
fig.add_trace(go.Scatter(
    x=types,
    y=[type_means[t] for t in types],
    mode='lines+markers',
    line=dict(color='rgba(0, 0, 0, 0.7)', width=8),
    marker=dict(
        size=14,  # Increased marker size
        color=[type_colors[t].replace('0.5', '1') for t in types],
        line=dict(color='black', width=4)
    ),
    name='Mean RT',
    hovertemplate='Mean: %{y:.3f}s<extra></extra>'
))

# Define a common font style for consistent text appearance
font_style = dict(
    family="Arial, sans-serif",
    size=20,  # Larger font size
    color="black"
)

# Update layout with enhanced text styling
fig.update_layout(
    title=dict(
        text="Average Reaction Times Across All Subjects\n\n",
        font=dict(size=25, family="Arial, sans-serif", color="black"),
        x=0.5,
        y=.985
    ),
    yaxis=dict(
        title=dict(
            text="Reaction Time (s)",
            font=font_style
        ),
        tickfont=font_style,
        tickwidth=2,
        showline=True,
        linewidth=3,
        linecolor='black'
    ),
    xaxis=dict(
        title=dict(
            text="Response Type",
            font=font_style
        ),
        tickmode='array',
        tickvals=types,
        tickfont=font_style,
        tickwidth=2,
        showline=True,
        linewidth=3,
        linecolor='black'
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.01,
        xanchor="center",
        x=0.5,
        font=dict(size=18, family="Arial, sans-serif"),
        borderwidth=2,
        bordercolor="White",
        title=dict(
            text="Response Type",
            font=dict(size=18, family="Arial, sans-serif")
        )
    ),
    violinmode='overlay',
    # height=1200,
    # width=1200,
    height=800,
    width=800,
    font=font_style,  # Default font for all other text elements
    plot_bgcolor='white',
    paper_bgcolor='white'
)

# Add annotations for the mean values
for i, type_name in enumerate(types):
    fig.add_annotation(
        x=type_name,
        y=type_means[type_name],
        text=f"{type_means[type_name]:.3f}s",
        showarrow=True,
        arrowhead=2,
        arrowsize=1.5,  # Slightly larger arrowhead
        arrowwidth=3,    # Thicker arrow
        arrowcolor='black',
        ax=120,
        ay=-90,
        font=dict(size=25, color='black', family="Arial, sans-serif")
    )

# Make the axis lines thicker
fig.update_xaxes(mirror=True, ticks='outside', tickwidth=3, ticklen=10, showgrid=False)
fig.update_yaxes(mirror=True, ticks='outside', tickwidth=3, ticklen=10, showgrid=True, gridwidth=1, gridcolor='lightgray')

fig.show()